This notebook is for peak matching and pivoting the df from long to wide format for ML

In [ ]:
import pandas as pd

df_main1 = pd.read_csv('Desktop/Jupyter Area/capstone-project/df_main1.csv')

# Checking for NAs and Nulls
print('CAS Null Count')
print(df_main1['CAS'].isnull().sum())
print('CAS NA Count')
print(df_main1['CAS'].isna().sum())
print('ID Null Count')
print(df_main1['ID'].isnull().sum())
print('ID NA Count')
print(df_main1['ID'].isna().sum())

# If there are, filter them out?
##################################################################################

In [ ]:
# Group peaks by CAS and RT values
    # same CAS might show up at different RT so they should be separate features
    #  Column bleed, isomer, co-elutions
df_temp = df_main1.sort_values(by=['CAS', 'RT']).reset_index(drop=True)

# Tolerance level from literature
rt_tolerance = 0.2


# assign a peak group within each CAS
def rt_group(group):
    rt_groups = []
    current_group = 0
    last_rt = None

    for rt in group['RT']:
        if last_rt is None or abs((rt - last_rt)) > rt_tolerance:
            current_group += 1
        rt_groups.append(current_group)
        last_rt = rt

    group['RT_Group'] = rt_groups
    return group


df_temp = df_temp.groupby('CAS', group_keys=False).apply(rt_group)  # Apply grouping by CAS

# Create unique PeakID (CAS+RT)
df_temp['Feature_ID'] = df_temp['CAS'] + '_group' + df_temp['RT_Group'].astype(str)

# Keep representative RTs for reporting
    # sure, you'll know the CAS, but it will just say group1 or group2 etc and you'll want to know the RT of that group
df_CAS_Group_LUT = df_temp.groupby(['Feature_ID', 'CAS'])['RT'].mean().reset_index()
df_CAS_Group_LUT.columns = ['Feature_ID', 'CAS', 'Mean_RT']

In [ ]:
# Pivot time
df_pivot = df_temp.pivot_table(
    index='PID',
    columns='Feature_ID',
    values='Area Pct',
    aggfunc='first'  # or 'mean' if duplicates
).reset_index()

# Merging Demographics back in
df_main2 = df_pivot.merge(
    df_main1[['PID', 'TB Status', 'Age', 'BMI', 'HIV Status']].drop_duplicates(),
    on='PID',
    how='left'
)

# Saving
df_main2.to_csv('df_main2.csv', index=False)
df_CAS_Group_LUT.to_csv('df_CAS_Group_LUT.csv', index=False)